In [1]:
import os

# Move up if inside notebooks/ folder
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

print("Installing dependencies from:", os.getcwd())
!pip install -r requirements.txt


Installing dependencies from: /home/sagemaker-user/SageNotebook


In [3]:
import getpass

# Detect if running inside SageMaker Studio
try:
    # In Studio, the default user is usually 'sagemaker-user'
    in_studio = getpass.getuser() == "sagemaker-user"
except Exception:
    in_studio = False

if in_studio:
    import sagemaker
    region = sagemaker.Session().boto_region_name
    role = sagemaker.get_execution_role()
    creds = {}
    print("✅ Running in SageMaker Studio")
else:
    from dotenv import load_dotenv
    load_dotenv(dotenv_path=".env", override=True)
    region = os.getenv("AWS_REGION")
    role = None
    creds = {
        "aws_access_key_id": os.getenv("AWS_ACCESS_KEY_ID"),
        "aws_secret_access_key": os.getenv("AWS_SECRET_ACCESS_KEY")
    }
    print("✅ Running locally, loaded .env")

print("Region:", region)


# Setup boto3 clients
runtime = boto3.client("sagemaker-runtime", region_name=region, **creds)
sm_client = boto3.client("sagemaker", region_name=region, **creds)




✅ Running in SageMaker Studio
Region: us-east-1


In [4]:
hub = {
    "HF_MODEL_ID": "distilbert-base-uncased-finetuned-sst-2-english",
    "HF_TASK": "text-classification"
}

print("✅ Hugging Face config ready")


✅ Hugging Face config ready


In [17]:
# --- Deploy (Studio only) ---
if in_studio:
    from sagemaker.huggingface import HuggingFaceModel
    import botocore

    endpoint_name = "huggingface-sentiment"

    # 🗑️ Clean up any old endpoint/config with the same name
    try:
        sm_client.delete_endpoint(EndpointName=endpoint_name)
        print(f"🗑️ Deleted old endpoint: {endpoint_name}")
    except botocore.exceptions.ClientError as e:
        if "Could not find endpoint" in str(e):
            print("No old endpoint found.")
        else:
            print("Delete endpoint warning:", e)

    try:
        sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
        print(f"🗑️ Deleted old endpoint config: {endpoint_name}")
    except botocore.exceptions.ClientError as e:
        if "Could not find endpoint configuration" in str(e):
            print("No old endpoint config found.")
        else:
            print("Delete endpoint config warning:", e)

    # Define HuggingFace model
    huggingface_model = HuggingFaceModel(
        transformers_version="4.26",
        pytorch_version="1.13",
        py_version="py39",
        env=hub,
        role=role
    )

    # Deploy new endpoint
    predictor = huggingface_model.deploy(
        initial_instance_count=1,
        instance_type="ml.m5.large",
        endpoint_name=endpoint_name
    )

    print("✅ Endpoint deployed:", endpoint_name)
else:
    print("⚠️ Skipping deployment (local mode)")


No old endpoint found.
🗑️ Deleted old endpoint config: huggingface-sentiment
------!✅ Endpoint deployed: huggingface-sentiment


In [18]:
if in_studio:
    print("🔍 Listing recent SageMaker endpoints...")
    endpoints = sm_client.list_endpoints(
        SortBy="CreationTime", SortOrder="Descending", MaxResults=10
    )
    for ep in endpoints["Endpoints"]:
        print(ep["EndpointName"], ep["EndpointStatus"])

    # Optional cleanup: delete timestamped ones
    for ep in endpoints["Endpoints"]:
        if ep["EndpointName"].startswith("huggingface-sentiment-"):
            sm_client.delete_endpoint(EndpointName=ep["EndpointName"])
            print("🗑️ Deleted:", ep["EndpointName"])
else:
    print("⚠️ Skipping endpoint listing/cleanup (local mode)")


🔍 Listing recent SageMaker endpoints...
huggingface-sentiment InService
huggingface-sentiment-1753733198 InService
🗑️ Deleted: huggingface-sentiment-1753733198


In [19]:
response = sm_client.list_endpoints(SortBy="CreationTime", SortOrder="Descending", MaxResults=5)
for ep in response["Endpoints"]:
    print(ep["EndpointName"], ep["EndpointStatus"])



huggingface-sentiment InService


In [22]:
import json

if in_studio:
    # Studio mode: safe to use predictor object
    result = predictor.predict({"inputs": "This SageMaker setup is fantastic!"})
    print("Prediction (Studio):", result)
else:
    # Local mode: use runtime.invoke_endpoint
    endpoint_name = os.getenv("SAGEMAKER_ENDPOINT_NAME")
    if not endpoint_name:
        # fallback: pick the latest InService endpoint
        endpoints = sm_client.list_endpoints(SortBy="CreationTime", SortOrder="Descending", MaxResults=1)
        if endpoints["Endpoints"]:
            endpoint_name = endpoints["Endpoints"][0]["EndpointName"]
            print(f"⚠️ Using latest endpoint: {endpoint_name}")
        else:
            raise ValueError("No SageMaker endpoint available!")

    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=json.dumps({"inputs": "This makes me angry and frustrated"})
    )

    print("Prediction (Local):", json.loads(response['Body'].read().decode()))




Prediction (Studio): [{'label': 'POSITIVE', 'score': 0.9998700618743896}]


In [24]:
sm_client.delete_endpoint(EndpointName="huggingface-sentiment")
print("Deleted old endpoint: huggingface-sentiment")


Deleted old endpoint: huggingface-sentiment
